# Decorator


In [1]:
def counter(fn):
    cnt = 0

    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        print(f"Function {fn.__name__} was called {cnt} times")
        return fn(*args, **kwargs)

    return inner

In [2]:
def add(a, b):
    return a + b


add = counter(add)

**What we are doing there?**
1. we are adding the extra functionality to the add function , the extra functionality is provided by the counter function.
2. We can also say we are decorating the add function using the decorator.
3. Extra functionality Provider  --> Decorator
4. Using the extra functionality --> Decorating the function

**Decorator**
1. Decorator function take the other function as the argument.
2. return the closure
3. the closure usually accept any combination of parameter
4. run the same code inside the inner function(closure)
5. the closure function call the original function using the argument passed to closere
6. return whatever is returned by that function call

<img src="image/img_1.png">

In [3]:
@counter
def mul(a, b):
    return a * b

In [4]:
mul(1, 2)

Function mul was called 1 times


2

In [5]:
mul(1, 2)

Function mul was called 2 times


2

In [6]:
mul(1, 2)

Function mul was called 3 times


2

# Introspecting Decorated Function

In [7]:
def mul(a: int, b: int, c: int = 1):
    """return the product of three values"""
    return a * b * c

In [8]:
help(mul)

Help on function mul in module __main__:

mul(a: int, b: int, c: int = 1)
    return the product of three values



In [9]:
@counter
def mul(a: int, b: int, c: int = 1):
    """return the product of three values"""
    return a * b * c

In [10]:
help(mul)
#! after we decorate teh function we lost the original metadata of the function

Help on function inner in module __main__:

inner(*args, **kwargs)



In [11]:
#? We may fix the function name and doc by copy to inner
def counter(fn):
    cnt = 0

    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        print(f"Function {fn.__name__} was called {cnt} times")
        return fn(*args, **kwargs)

    inner.__doc__ = fn.__doc__
    inner.__name__ = fn.__name__
    return inner


In [12]:
@counter
def mul(a: int, b: int, c: int = 1):
    """return the product of three values"""
    return a * b * c

In [14]:
help(mul)
#? we fix the function name and doc but didn't fix the signature of the function

Help on function mul in module __main__:

mul(*args, **kwargs)
    return the product of three values



In [15]:
#? to fix metadata loss of decorated function, python provide the warps
from functools import wraps


#* wraps also a decorator that copy the metadata from one function to another function
def counter(fn):
    cnt = 0

    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        print(f"Function {fn.__name__} was called {cnt} times")
        return fn(*args, **kwargs)

    return inner

In [16]:
@counter
def mul(a: int, b: int, c: int = 1):
    """return the product of three values"""
    return a * b * c


In [17]:
help(mul)

Help on function mul in module __main__:

mul(a: int, b: int, c: int = 1)
    return the product of three values



# Timer the Function

In [30]:
def timed(fn):
    from time import perf_counter
    from functools import wraps

    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start

        arg_ = [str(a) for a in args]
        kwargs_ = [f"{k}={v}" for k, v in kwargs.items()]
        all_args = arg_ + kwargs_
        args_str = ",".join(all_args)

        print(f"{fn.__name__}({args_str}) took {elapsed:.6f}s to run.")
        return result

    return inner


In [35]:
def cal_recursive_fib(n):
    if n <= 2:
        return 1
    return cal_recursive_fib(n - 1) + cal_recursive_fib(n - 2)

In [34]:
cal_recursive_fib(5)

cal_recursive_fib(2) took 0.000001s to run.
cal_recursive_fib(1) took 0.000002s to run.
cal_recursive_fib(3) took 0.000316s to run.
cal_recursive_fib(2) took 0.000001s to run.
cal_recursive_fib(4) took 0.000333s to run.
cal_recursive_fib(2) took 0.000000s to run.
cal_recursive_fib(1) took 0.000000s to run.
cal_recursive_fib(3) took 0.000014s to run.
cal_recursive_fib(5) took 0.000363s to run.


5

In [36]:
@timed
def fib_recursive(n):
    return cal_recursive_fib(n)

In [38]:
fib_recursive(6)

fib_recursive(6) took 0.000008s to run.


8

In [39]:
fib_recursive(20)

fib_recursive(20) took 0.000992s to run.


6765

In [40]:
fib_recursive(30)

fib_recursive(30) took 0.093450s to run.


832040

In [41]:
fib_recursive(36)

fib_recursive(36) took 1.748937s to run.


14930352

In [42]:
fib_recursive(40)

fib_recursive(40) took 11.599354s to run.


102334155

In [44]:
@timed
def fib_loop(n):
    a = b = 1
    for _ in range(3, n + 1):
        a, b = b, a + b
    return b

In [45]:
fib_loop(40)

fib_loop(40) took 0.000004s to run.


102334155

In [46]:
fib_loop(200)

fib_loop(200) took 0.000009s to run.


280571172992510140037611932413038677189525

In [47]:
fib_loop(10000)

fib_loop(10000) took 0.002611s to run.


3364476487643178326662161200510754331030214846068006390656476997468008144216666236815559551363373402558206533268083615937373479048386526826304089246305643188735454436955982749160660209988418393386465273130008883026923567361313511757929743785441375213052050434770160226475831890652789085515436615958298727968298751063120057542878345321551510387081829896979161312785626503319548714021428753269818796204693609787990035096230229102636813149319527563022783762844154036058440257211433496118002309120828704608892396232883546150577658327125254609359112820392528539343462090424524892940390170623388899108584106518317336043747073790855263176432573399371287193758774689747992630583706574283016163740896917842637862421283525811282051637029808933209990570792006436742620238978311147005407499845925036063356093388383192338678305613643535189213327973290813373264265263398976392272340788292817795358057099369104917547080893184105614632233821746563732124822638309210329770164805472624384237486241145309381220656491403

In [48]:
from functools import reduce

In [58]:
@timed
def fib_reduce(n):
    initial = (1,0)
    dummy = range(n)
    fib_n = reduce(lambda prev,_:(prev[0] + prev[1] ,prev[0]),
                   dummy,
                   initial)
    return fib_n[0]

In [59]:
fib_reduce(10000)

Running iteration 0
Running iteration 1
Running iteration 2
Running iteration 3
Running iteration 4
Running iteration 5
Running iteration 6
Running iteration 7
Running iteration 8
Running iteration 9
fib_reduce(10000) took 0.002490s to run.


5443837311356528133873426099375038013538918455469596702624771584120858286562234901708305154793896054117382267597802631738435958475111624143917470264295916992558633411790606304808979353147610846625907275936789915067796008830659796664196582493772180038144115884104248099798469648737533718002816376331778192794110136926275097950980071359671802381471066991264421477525447858767456896380800296226513311135992976272667944140010157580004351077746593580536250246170791805922641467900569075232189586814236784959388075642348375438634263963597073375626009896246266874611204173981940487506244370986865431562684718619562014612664223271181504036701882520531484587581719353352982783780035190252923951783668946766191795388471244102846393544948461445077876252952096188759727288922076853739647586954315917243453719361126374392633731300589616724805173798630636811500308839674958710261952463135244749950520419830518716832162328385979462724591977145462821839969578922379891219943177546970521613108109655995063829726125384

In [64]:
def timed(fn):
    from time import perf_counter
    from functools import wraps

    @wraps(fn)
    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(10):
            print(f"Running iteration {i}")
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            total_elapsed += end - start

        arg_ = [str(a) for a in args]
        kwargs_ = [f"{k}={v}" for k, v in kwargs.items()]
        all_args = arg_ + kwargs_
        args_str = ",".join(all_args)
        elapsed_avg = total_elapsed / 10

        print(f"{fn.__name__}({args_str}) took average {elapsed_avg:.6f}s to run.")
        return result

    return inner

In [65]:
@timed
def fib_reduce(n):
    initial = (1,0)
    dummy = range(n)
    fib_n = reduce(lambda prev,_:(prev[0] + prev[1] ,prev[0]),
                   dummy,
                   initial)
    return fib_n[0]

In [66]:
fib_reduce(1000)

Running iteration 0
Running iteration 1
Running iteration 2
Running iteration 3
Running iteration 4
Running iteration 5
Running iteration 6
Running iteration 7
Running iteration 8
Running iteration 9
fib_reduce(1000) took average 0.000211s to run.


70330367711422815821835254877183549770181269836358732742604905087154537118196933579742249494562611733487750449241765991088186363265450223647106012053374121273867339111198139373125598767690091902245245323403501